In [ ]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG, display
# import shap

import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
# from pycaret.classification import *
import lightgbm as lgb
from sklearn.metrics import recall_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize, space
from functools import partial
from sklearn.metrics import average_precision_score

In [ ]:
df = pd.read_csv('/content/sample_data/Best_dataset_fp.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# Split the DataFrame into training set (90%) and test set (10%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
X

NameError: ignored

In [ ]:
Xt = train_df.drop(['TenYearCHD'], axis = 1)
yt = train_df['TenYearCHD']

smote = SMOTE(random_state = 8)
X_smote, y_smote = smote.fit_resample(Xt, yt)

In [ ]:


def optimize(params, X, y):
    params = dict(zip(param_names, params))
    model = lgb.LGBMClassifier(**params)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        model.fit(X_train, y_train, early_stopping_rounds=10,
                  eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict_proba(X_val)[:, 1]
        score = average_precision_score(y_val, y_pred)
        scores.append(score)
    return -1.0 * np.mean(scores)
# Define the parameter space and names
param_space = [
    space.Integer(3, 15, name="max_depth"),
    space.Integer(100, 600, name="n_estimators"),
    space.Real(0.01, 1, prior="uniform", name="learning_rate"),
    space.Real(0.1, 1, prior="uniform", name="subsample"),
    space.Real(0.1, 1, prior="uniform", name="colsample_bytree"),
    space.Real(0.1, 10, prior="uniform", name="min_child_weight")
    ]
param_names = ["max_depth", "n_estimators", "learning_rate", "subsample", "colsample_bytree", "min_child_weight"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X_smote, y=y_smote)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=50,
    n_random_starts=50,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(f'Best Hyperparameters: {dict(zip(param_names, result.x))}')
print(f'Best AUPRC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.0695
Function value obtained: -0.9268
Current minimum: -0.9268
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.7201
Function value obtained: -0.9553
Current minimum: -0.9553
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.5027
Function value obtained: -0.9436
Current minimum: -0.9553
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.4158
Function value obtained: -0.9523
Current minimum: -0.9553
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.1321
Function value obtained: -0.9468
Current minimum: -0.9553
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'max_depth': 15, 'n_estimators': 369, 'learning_rate': 0.16950148906161272,
          'subsample': 0.929310395457466, 'colsample_bytree': 0.7018351167249548, 'min_child_weight': 4.9739379442117215}



# Train the model
model = lgb.LGBMClassifier(**params)
model.fit(X_smote, y_smote)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUPRC
auprc = average_precision_score(y_test, y_pred)
print("AUPRC:", auprc)

AUPRC: 0.2144363965777155


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = GradientBoostingClassifier(**params, n_iter_no_change=10)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)
# Define the parameter space and names

param_space = [
    Real(0.01, 1, prior='log-uniform', name='learning_rate'),
    Integer(2, 10, name='max_depth'),
    Real(0.1, 1, prior='uniform', name='subsample'),
    Real(0.1, 1, prior='uniform', name='max_features'),
    Integer(2, 100, name='n_estimators'),
    Integer(2, 50, name='min_samples_split'),
    Integer(1, 20, name='min_samples_leaf')
]

param_names = ["learning_rate", "max_depth", "subsample", "max_features", "n_estimators", "min_samples_split", "min_samples_leaf"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X_smote, y=y_smote)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=20,
    n_random_starts=20,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(f'Best Hyperparameters: {dict(zip(param_names, result.x))}')
print(f'Best AUPRC Score: {-1.0 * result.fun}')


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 4.7770
Function value obtained: -0.8622
Current minimum: -0.8622
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.9226
Function value obtained: -0.8038
Current minimum: -0.8622
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7799
Function value obtained: -0.6985
Current minimum: -0.8622
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.2782
Function value obtained: -0.7911
Current minimum: -0.8622
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 5.5696
Function value obtained: -0.8559
Current minimum: -0.8622
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'learning_rate': 0.08514461365334756, 'max_depth': 8, 'subsample': 0.9433614750342277,
          'max_features': 0.9953293863937087, 'n_estimators': 73, 'min_samples_split': 16, 'min_samples_leaf': 4}


# Train the model
model = GradientBoostingClassifier(**params)
model.fit(X_smote, y_smote)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auprc = average_precision_score(y_test, y_pred)
print("AUPRC:", auprc)


AUPRC: 0.2463922337899996
